In [1]:
#MonteCarlo Time of Arrival GPS Sim for Receiver Localization 
#Fausto Vega

using SatelliteDynamics
using LinearAlgebra
using PlotlyJS
using DelimitedFiles
using Distributions
using ForwardDiff
using BlockDiagonals
using Downloads
#using GeoMakie, Downloads
#using FileIO
#using RPRMakie

WebIO._IJuliaInit()

In [2]:
#custom scale
distance_scale = R_EARTH*(1e-3) # scales km to custom scale
time_scale = 1/(C_LIGHT/R_EARTH) #scales seconds to custom scale

c = 1 # 1 distance_unit/ 1 time_unit

1

In [3]:
# Declare simulation initial Epoch. Start time
epc0 = Epoch(2019, 1, 1, 12, 0, 0, 0.0)  #year, month, day, hour, minute, seconds, nanoseconds 

Epoch(2019-01-01T11:59:23.000Z)

In [4]:
# Declare initial state in terms of osculating orbital elements
#semi-major axis, eccentricity, inclination, RAAN, Argument of perigee, Mean anomaly

#1 and 2 on the same orbit, different initial mean anomaly

#previous configuration (working but not 100 km apart at the equator) (hard to do in practice)
#iss1 = [7378e3, 0.0004879, 90.6391, 189.0859, 151.2014, 185]; #converges at 189.0859
#iss2 = [7378e3, 0.0004879, 90.6391, 189.0859, 151.2014, 205];
#iss3 = [7378e3, 0.0004879, 90.6391, 201.0859, 151.2014, 190]; # converges a t 200.0859
#iss4 = [7378e3, 0.0004879, 90.6391, 201.0859, 151.2014, 215]; # converges a t 200.0859

#approximately 500 km elevation from surface of Earth and around 100 km seperation at equator
#working good
iss1 = [6871e3, 0.0004879, 90.6391, 194.0859, 151.2014, 190]; 
iss2 = [6871e3, 0.0004879, 90.6391, 194.0859, 151.2014, 200];
iss3 = [6871e3, 0.0004879, 90.6391, 196.0859, 151.2014, 193]; 
iss4 = [6871e3, 0.0004879, 90.6391, 196.0859, 151.2014, 203]; 

E1 = anomaly_mean_to_eccentric(iss1[6], iss1[2], use_degrees=true);
E2 = anomaly_mean_to_eccentric(iss2[6], iss2[2], use_degrees=true);
E3 = anomaly_mean_to_eccentric(iss3[6], iss3[2], use_degrees=true);
E4 = anomaly_mean_to_eccentric(iss4[6], iss4[2], use_degrees=true);

In [5]:
# Convert osculating elements to Cartesean state
# returns position and velocity (m, m/s)

eci0_1 = sOSCtoCART(iss1, use_degrees=true)
eci0_2 = sOSCtoCART(iss2, use_degrees=true)
eci0_3 = sOSCtoCART(iss3, use_degrees=true)
eci0_4 = sOSCtoCART(iss4, use_degrees=true)

6-element Vector{Float64}:
      -6.568741989057123e6
      -1.9023114030561317e6
 -697065.2121487064
    -763.850262000727
    -132.345846649287
    7573.5676513547705

In [6]:
#compute the satellite orbital period given the semi-major axis

#same for all 4 satellites since they have the same semi-major axis
T = orbit_period(iss1[1])

#final time for one orbit. Adds initial time to the orbital period

epcf = epc0 + T

Epoch(2019-01-01T13:33:51.144Z)

In [7]:
# Create an EarthInertialState orbit propagagator
#needs initial epoch of state and the state vector
orb1  = EarthInertialState(epc0, eci0_1, dt=1.0,
            mass=1.0, n_grav=0, m_grav=0,
            drag=false, srp=false,
            moon=false, sun=false,
            relativity=false
)

orb2  = EarthInertialState(epc0, eci0_2, dt=1.0,
            mass=1.0, n_grav=0, m_grav=0,
            drag=false, srp=false,
            moon=false, sun=false,
            relativity=false
)

orb3  = EarthInertialState(epc0, eci0_3, dt=1.0,
            mass=1.0, n_grav=0, m_grav=0,
            drag=false, srp=false,
            moon=false, sun=false,
            relativity=false
)

orb4  = EarthInertialState(epc0, eci0_4, dt=1.0,
            mass=1.0, n_grav=0, m_grav=0,
            drag=false, srp=false,
            moon=false, sun=false,
            relativity=false
)

# Propagate the orbit
# orbit until the final time

t_1, epc_1, eci_1 = sim!(orb1, epcf);
t_2, epc_2, eci_2 = sim!(orb2, epcf);
t_3, epc_3, eci_3 = sim!(orb3, epcf);
t_4, epc_4, eci_4 = sim!(orb4, epcf);

In [8]:
#equator position in cartesian coordinates
#tag =[-6.128804e6, -1.590206e6, 776.1502e3]
#Equitorial position
tag_geof = [-165.4545, 0, 0]

#North pole position
#tag_geof = [-165.4545, 46.98849, 0]

tag = sGEOCtoECEF(tag_geof, use_degrees = true)

3-element Vector{Float64}:
 -6.173708155673107e6
 -1.6018611673126891e6
  0.0

In [9]:
#Equitorial Position
#In m
x0= tag[1] 
y0 = tag[2]
z0 = tag[3]
t0 = 1e-5

r0 = [x0, y0, z0, t0]

centroid_guess = [(eci_1[1,1]+eci_2[1,1]+eci_3[1,1]+eci_4[1,1])/4, (eci_1[2,1]+eci_2[2,1]+eci_3[2,1]+eci_4[2,1])/4, (eci_1[3,1]+eci_2[3,1]+eci_3[3,1]+eci_4[3,1])/4] 
onearth = sECEFtoGEOC(centroid_guess, use_degrees = true)
geodetic = [onearth[1], onearth[2], 0]

#Guess
xyz = sGEOCtoECEF(geodetic, use_degrees = true)

println(xyz)

[-6.012480777483758e6, -1.6370620464816242e6, -1.3604169754625652e6]


In [10]:
combinedeci = [eci_1; eci_2;eci_3; eci_4]

println(size(combinedeci))

println(eci_2[:,1])

(24, 5670)
[-6.585795034854666e6, -1.664632559920686e6, -1.0536819473539353e6, -1151.1334736314636, -202.32620515831618, 7522.817373459283]


In [11]:
#Plot Satellite 1 and 2 Orbit
sat1 = scatter(x=eci_1[1,:], y=eci_1[2,:], z=eci_1[3,:], type="scatter3d", mode="lines", name="orbit 1&2")

#Plot Satellite 3 and 4 Orbit
sat3 = scatter(x=eci_3[1,:], y=eci_3[2,:], z=eci_3[3,:], type="scatter3d", mode="lines", name="orbit 3&4")

#Plot all 4 satellite initial positions
satellites = scatter(x=[eci_1[1,1],eci_2[1,1], eci_3[1,1], eci_4[1,1]], y=[eci_1[2,1],eci_2[2,1], eci_3[2,1], eci_4[2,1]],z=[eci_1[3,1],eci_2[3,1], eci_3[3,1], eci_4[3,1]], mode="markers", marker_size = 4, type="scatter3d", name="satellites")

#Plot Tag position
tag = scatter(x = [r0[1], r0[1]], y = [r0[2], r0[2]], z = [r0[3], r0[3]], type="scatter3d", name="tag", mode="markers", marker_size=5)

#Plot the Guess
guess = scatter(x = [xyz[1], xyz[1]], y = [xyz[2], xyz[2]], z = [xyz[3], xyz[3]], type="scatter3d", name="guess", mode="markers", marker_size=5)

#Plotting the sphere (Earth)
n = 100
u = range(-π, π; length = n)
v = range(0, π; length = n)
x = cos.(u) * sin.(v)'
y = sin.(u) * sin.(v)' 
z = ones(n) * cos.(v)' 

#earth_img = load(Downloads.download("https://upload.wikimedia.org/wikipedia/commons/5/56/Blue_Marble_Next_Generation_%2B_topography_%2B_bathymetry.jpg"))

earth = surface(z=z*6371000, x=x*6371000, y=y*6371000, showscale = false)

plot([sat1,sat3, satellites, tag, earth, guess])

data: [
  "scatter3d with fields mode, name, type, x, y, and z",
  "scatter3d with fields mode, name, type, x, y, and z",
  "scatter3d with fields marker, mode, name, type, x, y, and z",
  "scatter3d with fields marker, mode, name, type, x, y, and z",
  "surface with fields showscale, type, x, y, and z",
  "scatter3d with fields marker, mode, name, type, x, y, and z"
]

layout: "layout with fields margin and template"

In [12]:
norm(eci_1[1,1] - eci_3[1,1])

43871.78133845888

In [13]:
#Finding the Elevation Angle

function zenith_angle(satposes, r0)
    
    zenithangle = zeros(4)

    #vector between tag and the satellite
    normalvec = r0
    
    for i in 1:4
        
        vector = satposes[i,:] - r0[1:3]
        
        #Find the angle between the normal and the vector going from the tag to the satellte
        theta = acos(dot(normalvec,vector)/(norm(normalvec)*norm(vector))) * (180/pi)
        
        #save elevation angle
        zenithangle[i] = theta
    end
    
    return zenithangle
end

zenith_angle (generic function with 1 method)

In [14]:

#Testing elevation angle function
satposes = [eci_1[1,1] eci_1[2,1] eci_1[3,1];eci_2[1,1] eci_2[2,1] eci_2[3,1];eci_3[1,1] eci_3[2,1] eci_3[3,1];eci_4[1,1] eci_4[2,1] eci_4[3,1]]

zenith_angles = zenith_angle(satposes, r0[1:3])

println(zenith_angles)

#check all angles to see if it is on the horizon 
inhorizon = all(x->x<80, zenith_angles)

[86.66674618087028, 68.53546110136095, 82.94499684820703, 57.653650047965165]


false

In [15]:
#Obtain the orbital elements after each time step 
dim = size(t_1)[1]

all_elements = zeros(6,dim)
for i in 1:dim
    
    orbital_elements = sCARTtoOSC(eci_1[:,1], use_degrees=true)
    all_elements[:,i] = orbital_elements
end 

In [16]:
#plot(dim, all_elements[1,:], xlabel = "t (s)", ylabel = "a [km]", title = "Semi-major Axis",  ylims = (6.2e6, 7.4e6))

In [17]:
#plt = plot3d(1, xaxis=("x", (-6e6, 6e6)), yaxis=("y", (-5e6, 5e6)), zaxis=("z", (-5e6, 5e6)), title="ISS Orbit", marker=2, markercolor="black")

#anim = @animate for i in 1:dim
#    push!(plt, eci_1[1,i], eci_1[2,i], eci_1[3,i])
#    push!(plt, eci_2[1,i], eci_2[2,i], eci_2[3,i])
#    push!(plt, eci_3[1,i], eci_3[2,i], eci_3[3,i])
#    push!(plt, eci_4[1,i], eci_4[2,i], eci_4[3,i])
#end

In [18]:
#gif(anim, "orbit.gif", fps=1000)

In [19]:
#Generate TEC Distribution for night TEC values
mu = 8e16
sigma = 3e16
lb = 3e16
ub = 13e16
d = Truncated(Normal(mu,sigma), lb, ub)

#Generate TEC Distribution for day TEC values
#mu = 35e16
#sigma = 5e16
#lb = 30e16
#ub = 40e16
#d = Truncated(Normal(mu,sigma), lb, ub)

#f = 1575.42e6 # in 1/s L1 frequency
f = 400e6 # in 1/s system frequency
hi = 350e3 #in meters
Ip = zeros(4)
OF = zeros(4)
Iz = zeros(4)
Ip_scaled = zeros(4)

4-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0

In [20]:
dunit = 1/distance_scale
tunit = 1/time_scale
speed = dunit/tunit

3.33564095198152e-6

In [21]:
function measurment_residual(r0, r1, r2, r3, r4, t)
    
    omega = OMEGA_EARTH*time_scale
    
    #pass in as column vectors
    res1 = (norm(r0[1:3] - ([cos(omega*t[1]) sin(omega*t[1]) 0;
          -sin(omega*t[1]) cos(omega*t[1]) 0;
          0 0 1]) * r1[1:3]))/c + r0[4] - t[1]
    res2 = (norm(r0[1:3] - ([cos(omega*t[2]) sin(omega*t[2]) 0;
          -sin(omega*t[2]) cos(omega*t[2]) 0;
          0 0 1])* r2[1:3]))/c + r0[4] - t[2] 
    res3 = (norm(r0[1:3] - ([cos(omega*t[3]) sin(omega*t[3]) 0;
          -sin(omega*t[3]) cos(omega*t[3]) 0;
          0 0 1])*r3[1:3]))/c + r0[4] - t[3] 
    res4 = (norm(r0[1:3] - ([cos(omega*t[4]) sin(omega*t[4]) 0;
          -sin(omega*t[4]) cos(omega*t[4]) 0;
          0 0 1])*r4[1:3]))/c + r0[4] - t[4]
    
    return [res1, res2, res3, res4]
end

measurment_residual (generic function with 1 method)

In [22]:
function get_time(r0_scaled, r1, r2, r3, r4)
    
    n = 1000

    time = [zeros(4) for i = 1:1000]
    Rt = [zeros(4) for i = 1:1000]

    time[1] = [0.1\time_scale, 0.1\time_scale, 0.1\time_scale, 0.1\time_scale] #initial guess

    Rt[1] = measurment_residual(r0_scaled, r1, r2, r3, r4, time[1])

    iters = 0

    for i=1:n

        Rt[i] = measurment_residual(r0_scaled, r1, r2, r3, r4, time[i])

        iters += 1

        if(norm(Rt[i]) < 1e-6)
            break
        end

        jacobian = ForwardDiff.jacobian(dt -> measurment_residual(r0_scaled, r1, r2, r3, r4, dt), time[i])

        deltat = (jacobian)\-Rt[i]

        time[i+1]  = time[i] + deltat
    end
    
    Rt = Rt[1:iters]
    time = time[1:iters]
    time_measurment = time[end]
    
    return time_measurment
    
end

get_time (generic function with 1 method)

In [23]:
function residual(x, sat_pose_t)
    
    omega = OMEGA_EARTH*time_scale
 
    A1 = [cos(omega*sat_pose_t[1,4]) sin(omega*sat_pose_t[1,4]) 0;
          -sin(omega*sat_pose_t[1,4]) cos(omega*sat_pose_t[1,4]) 0;
          0 0 1];
    A2 = [cos(omega*sat_pose_t[2,4]) sin(omega*sat_pose_t[2,4]) 0;
          -sin(omega*sat_pose_t[2,4]) cos(omega*sat_pose_t[2,4]) 0;
          0 0 1];
    A3 = [cos(omega*sat_pose_t[3,4]) sin(omega*sat_pose_t[3,4]) 0;
          -sin(omega*sat_pose_t[3,4]) cos(omega*sat_pose_t[3,4]) 0;
          0 0 1];
    A4 = [cos(omega*sat_pose_t[4,4]) sin(omega*sat_pose_t[4,4]) 0;
          -sin(omega*sat_pose_t[4,4]) cos(omega*sat_pose_t[4,4]) 0;
          0 0 1];
    
    res1 = norm(x[1:3] - A1*sat_pose_t[1,1:3]) - c*(sat_pose_t[1,4] - x[4]);
    res2 = norm(x[1:3] - A1*sat_pose_t[2,1:3]) - c*(sat_pose_t[2,4] - x[4]);
    res3 = norm(x[1:3] - A1*sat_pose_t[3,1:3]) - c*(sat_pose_t[3,4] - x[4]);
    
    res4 = norm(x[1:3] - A2*sat_pose_t[4,1:3]) - c*(sat_pose_t[4,4] - x[4]);
    res5 = norm(x[1:3] - A2*sat_pose_t[5,1:3]) - c*(sat_pose_t[5,4] - x[4]);
    res6 = norm(x[1:3] - A2*sat_pose_t[6,1:3]) - c*(sat_pose_t[6,4] - x[4]);
    
    res7 = norm(x[1:3] - A3*sat_pose_t[7,1:3]) - c*(sat_pose_t[7,4] - x[4]);
    res8 = norm(x[1:3] - A3*sat_pose_t[8,1:3]) - c*(sat_pose_t[8,4] - x[4]);
    res9 = norm(x[1:3] - A3*sat_pose_t[9,1:3]) - c*(sat_pose_t[9,4] - x[4]);
    
    res10 = norm(x[1:3] - A4*sat_pose_t[10,1:3]) - c*(sat_pose_t[10,4] - x[4]);
    res11 = norm(x[1:3] - A4*sat_pose_t[11,1:3]) - c*(sat_pose_t[11,4] - x[4]);
    res12 = norm(x[1:3] - A4*sat_pose_t[12,1:3]) - c*(sat_pose_t[12,4] - x[4]);
    
    residuals = [res1, res2, res3, res4, res5, res6, res7, res8, res9, res10, res11, res12]
    
    return residuals
end

residual (generic function with 1 method)

In [24]:
#Equitorial Position

#In km
x0= x0*1e-3
y0 = y0*1e-3
z0 = z0*1e-3
t0 = 1e-5

#r0 = [x0,y0,z0,t0]
r0_scaled = [x0/distance_scale, y0/distance_scale, z0/distance_scale, t0/time_scale]

4-element Vector{Float64}:
 -0.9679486083847263
 -0.2511487826487322
  0.0
  0.0004700314384940316

In [25]:
#clock error distribution
mu_t = 0
sigma_t = 20e-9 #RMSE for Novatel GPS receiver
lb_t = 1e-9
ub_t = 30e-9
#d_t = Truncated(Normal(mu_t,sigma_t), lb_t, ub_t)
d_t = Normal(mu_t, sigma_t)

Normal{Float64}(μ=0.0, σ=2.0e-8)

In [26]:
function receiver_pose(all_sats_scaled, guess, zenith_angles) # remember to scale the guess to custom units

    n = 1000 # number of iterations
    #all_r0 = NaN*[zeros(4) for i = 1:n]
    all_r0 = [zeros(4) for i = 1:n]
    
    
    sat1poses = zeros(4,n)
    sat2poses = zeros(4,n)
    sat3poses = zeros(4,n)
    sat4poses = zeros(4,n)

    all_sats_noise = zeros(4,4)

    iters = 0

     #Monte Carlo Simulation
    for i in 1:n
        b = 0.01
        c=0.5
        β = 10
        α = 1
        
        #create noise from random distribution
        #scaled to the variable (0.1m error for distance & 1e-11 for time)
        gpsnoise = randn(12)*(1e-4/distance_scale) #adding a 0.1 meter of noise
        #clocknoise = randn(4)*(1e-11/time_scale)
        clocknoise = rand(d_t,4)/time_scale
        #iterations = 0
        
        TEC = rand(d,4) #vTEC for each satellite from custom distribution
        
        #println("this is tEC: ", TEC)
        
        #Calculating random TEC time delay
        for i=1:4
        
            OF[i] = (1-((R_EARTH*sind(zenith_angles[i]))/(R_EARTH+hi))^2)^(-1/2) #normal units (m and s)
            Ip[i] = ((40.3*TEC[i])/(f^2))*OF[i] * 1e-3 #scale to km 
            Ip_scaled[i] = Ip[i]/distance_scale #scale to custom units
        
        end
        
        sat1_1_noise = [all_sats_scaled[1,1]+gpsnoise[1],all_sats_scaled[1,2] + gpsnoise[2],all_sats_scaled[1,3]+gpsnoise[3], all_sats_scaled[1,4] + clocknoise[1]+ (Ip_scaled[1]/c)]
        sat1_2_noise = [all_sats_scaled[2,1]+gpsnoise[1],all_sats_scaled[2,2] + gpsnoise[2],all_sats_scaled[2,3]+gpsnoise[3], all_sats_scaled[2,4] + clocknoise[1]+ (Ip_scaled[1]/c)]
        sat1_3_noise = [all_sats_scaled[3,1]+gpsnoise[1],all_sats_scaled[3,2] + gpsnoise[2],all_sats_scaled[3,3]+gpsnoise[3], all_sats_scaled[3,4] + clocknoise[1]+ (Ip_scaled[1]/c)]

        sat2_1_noise =[all_sats_scaled[4,1]+gpsnoise[4],all_sats_scaled[4,2] + gpsnoise[5],all_sats_scaled[4,3]+gpsnoise[6], all_sats_scaled[4,4] + clocknoise[2]+ (Ip_scaled[2]/c)]
        sat2_2_noise = [all_sats_scaled[5,1]+gpsnoise[4],all_sats_scaled[5,2] + gpsnoise[5],all_sats_scaled[5,3]+gpsnoise[6], all_sats_scaled[5,4] + clocknoise[2]+ (Ip_scaled[2]/c)]
        sat2_3_noise = [all_sats_scaled[6,1]+gpsnoise[4],all_sats_scaled[6,2] + gpsnoise[5],all_sats_scaled[6,3]+gpsnoise[6], all_sats_scaled[6,4] + clocknoise[2]+ (Ip_scaled[2]/c)]

        sat3_1_noise = [all_sats_scaled[7,1]+gpsnoise[7],all_sats_scaled[7,2] + gpsnoise[8],all_sats_scaled[7,3]+gpsnoise[9], all_sats_scaled[7,4] + clocknoise[3]+ (Ip_scaled[3]/c)]
        sat3_2_noise = [all_sats_scaled[8,1]+gpsnoise[7],all_sats_scaled[8,2] + gpsnoise[8],all_sats_scaled[8,3]+gpsnoise[9], all_sats_scaled[8,4] + clocknoise[3]+ (Ip_scaled[3]/c)]
        sat3_3_noise = [all_sats_scaled[9,1]+gpsnoise[7],all_sats_scaled[9,2] + gpsnoise[8],all_sats_scaled[9,3]+gpsnoise[9], all_sats_scaled[9,4] + clocknoise[3]+ (Ip_scaled[3]/c)]
        
        sat4_1_noise = [all_sats_scaled[10,1]+gpsnoise[10],all_sats_scaled[10,2] + gpsnoise[11],all_sats_scaled[10,3]+gpsnoise[12], all_sats_scaled[10,4] + clocknoise[4]+ (Ip_scaled[4]/c)]
        sat4_2_noise = [all_sats_scaled[11,1]+gpsnoise[10],all_sats_scaled[11,2] + gpsnoise[11],all_sats_scaled[11,3]+gpsnoise[12], all_sats_scaled[11,4] + clocknoise[4]+ (Ip_scaled[4]/c)]
        sat4_3_noise = [all_sats_scaled[12,1]+gpsnoise[10],all_sats_scaled[12,2] + gpsnoise[11],all_sats_scaled[12,3]+gpsnoise[12], all_sats_scaled[12,4] + clocknoise[4]+ (Ip_scaled[4]/c)]
        
        all_sats_noise = vcat(sat1_1_noise',sat1_2_noise',sat1_3_noise',sat2_1_noise',sat2_2_noise',sat2_3_noise',sat3_1_noise',sat3_2_noise',sat3_3_noise',sat4_1_noise',sat4_2_noise',sat4_3_noise')
        
        #all_sats_noise[:,:] = vcat(sat1_noise',sat2_noise',sat3_noise',sat4_noise') #working

        #sat1poses[:,i] = sat1_noise
        #sat2poses[:,i] = sat2_noise
        #sat3poses[:,i] = sat3_noise
        #sat4poses[:,i] = sat4_noise

        #X = NaN*[zeros(4) for i = 1:100]
        #R = NaN*[zeros(4) for i = 1:100]
        
        X = NaN*[zeros(4) for i = 1:1000]
        R = NaN*[zeros(12) for i = 1:1000]
        
        #print(size(guess))
        #really close initial guess
        #X[1] = [(x0+100)/distance_scale, (y0+100)/distance_scale, (z0+100)/distance_scale, 0.0001/time_scale]
        #Initial guess 1000 km away
        #X[1] = [guess[1:3]/distance_scale 0.01/time_scale]
        
        #Centroid guess
        X[1] = [guess[1]/distance_scale, guess[2]/distance_scale, guess[3]/distance_scale, 0.0001/time_scale]


        #Initial residual
        R[1] = residual(X[1], all_sats_noise)

        iters = 0

        for k=1:1000
            R[k] = residual(X[k], all_sats_noise)
            
            #println("this is residual: ", norm(R[k]))

            iters += 1
        
            #original tolerancec 1e-12
            
            if(norm(R[k]) < 1e-6)
                #print("residual low")

                break

            end

            jacobian = ForwardDiff.jacobian(dx -> residual(dx, all_sats_noise), X[k])

            conditionnum = cond(jacobian)
            
            #println("this is the jacobian: ", jacobian)
            #println("condition nubmer: ", conditionnum)

            deltax = (jacobian)\-R[k]
            
            #println("this is deltax: ", deltax)
            
            while norm(residual(X[k] + α*deltax, all_sats_noise)) > norm(residual(X[k], all_sats_noise) + b*α*jacobian*deltax)

                α = c*α
                #print("this is alpha: ", α)
            end

            X[k+1] = X[k] + α*deltax
            
        end  
        
        R = R[1:iters]
        X = X[1:iters]

        all_r0[i] = X[end]

    end
    x_values = zeros(n)
    y_values = zeros(n)
    z_values = zeros(n)

    for j in 1:n

        x_values[j] = all_r0[j][1]
        y_values[j] = all_r0[j][2]
        z_values[j] = all_r0[j][3]

    end
    
    #mean calculation
    mean = sum(all_r0, dims = 1)/ n
    mean = mean[1]
    mean_rescaled = [mean[1:3]*distance_scale*1e3; mean[4]*time_scale] # in meters
    #print("done")
    return mean_rescaled, all_r0, iters
end

receiver_pose (generic function with 1 method)

In [27]:
size(eci_1)[2]

5670

In [28]:
#Navigation
#count = 0
n=1000
PDOP_array = zeros(size(eci_1)[2],1)
for i=1:size(eci_1)[2]
    
    #TEC = rand(d,4) #vTEC for each satellite from custom distribution
    
    satposes = [eci_1[1,i] eci_1[2,i] eci_1[3,i];eci_2[1,i] eci_2[2,i] eci_2[3,i];eci_3[1,i] eci_3[2,i] eci_3[3,i];eci_4[1,i] eci_4[2,i] eci_4[3,i]]
    
    zenith_angles = zenith_angle(satposes, r0[1:3])

    #check all angles to see if it is on the horizon 
    
    inhorizon = all(x->x<70, zenith_angles)
    
    if inhorizon==false
        #PDOP_array[i,1] = NaN
        continue #skip the current iteration
    end
#i=1
    #count = count+1

    #In km then to custom units
    r1_1 = [eci_1[1,i], eci_1[2,i], eci_1[3,i], 0]*1e-3/distance_scale
    r2_1 = [eci_2[1,i], eci_2[2,i], eci_2[3,i], 0]*1e-3/distance_scale
    r3_1 = [eci_3[1,i], eci_3[2,i], eci_3[3,i], 0]*1e-3/distance_scale
    r4_1 = [eci_4[1,i], eci_4[2,i], eci_4[3,i], 0]*1e-3/distance_scale
    
    t1 = get_time(r0_scaled, r1_1, r2_1, r3_1, r4_1)
    
    r1_1[4] = t1[1]
    r2_1[4] = t1[2]
    r3_1[4] = t1[3]
    r4_1[4] = t1[4]
    
    #In km then to custom units
    r1_2 = [eci_1[1,i]+3, eci_1[2,i], eci_1[3,i], 0]*1e-3/distance_scale
    r2_2 = [eci_2[1,i]+3, eci_2[2,i], eci_2[3,i], 0]*1e-3/distance_scale
    r3_2 = [eci_3[1,i]+3, eci_3[2,i], eci_3[3,i], 0]*1e-3/distance_scale
    r4_2 = [eci_4[1,i]+3, eci_4[2,i], eci_4[3,i], 0]*1e-3/distance_scale
    
    t2 = get_time(r0_scaled, r1_2, r2_2, r3_2, r4_2)
    
    r1_2[4] = t2[1]
    r2_2[4] = t2[2]
    r3_2[4] = t2[3]
    r4_2[4] = t2[4]
    
    #In km then to custom units
    r1_3 = [eci_1[1,i]-3, eci_1[2,i], eci_1[3,i], 0]*1e-3/distance_scale
    r2_3 = [eci_2[1,i]-3, eci_2[2,i], eci_2[3,i], 0]*1e-3/distance_scale
    r3_3 = [eci_3[1,i]-3, eci_3[2,i], eci_3[3,i], 0]*1e-3/distance_scale
    r4_3 = [eci_4[1,i]-3, eci_4[2,i], eci_4[3,i], 0]*1e-3/distance_scale
    
    t3 = get_time(r0_scaled, r1_3, r2_3, r3_3, r4_3)
    
    r1_3[4] = t3[1]
    r2_3[4] = t3[2]
    r3_3[4] = t3[3]
    r4_3[4] = t3[4]
    
    #all_sats_scaled = vcat(r1',r2',r3',r4')
    
    all_sats_scaled = vcat(r1_1',r1_2',r1_3',r2_1',r2_2',r2_3',r3_1',r3_2',r3_3',r4_1',r4_2',r4_3')
    
    #Generate a guess at the centroid of all 4 sats on the surface of the Earth. Scaled to km
    
    centroid_guess = [(eci_1[1,i]+eci_2[1,i]+eci_3[1,i]+eci_4[1,i])/4, (eci_1[2,i]+eci_2[2,i]+eci_3[2,i]+eci_4[2,i])/4, (eci_1[3,i]+eci_2[3,i]+eci_3[3,i]+eci_4[3,i])/4] 
    #centroid_guess = eci_2[1:3, i]
    onearth = sECEFtoGEOC(centroid_guess, use_degrees = true)
    geodetic = [onearth[1], onearth[2], 0]

    xyz_guess = sGEOCtoECEF(geodetic, use_degrees = true)*1e-3 #change to km

    #Find all possible positions from montecarlo and send unscaled guess
    mean_rescaled, all_r0, iters = receiver_pose(all_sats_scaled, xyz_guess, zenith_angles)
    
    all_r0_scaled = zeros(n,4)

    #Rescale back to units (m and s)
    for i in 1:n
    
        all_r0_scaled[i,:] = [all_r0[i][1:3]*distance_scale*1e3; all_r0[i][4]*time_scale]
        
    end

    totalsum = zeros(4,4)

    for i in 1:n
    
        value = all_r0_scaled[i,:] - mean_rescaled
        matrixvalue = value*transpose(value)
        totalsum += matrixvalue
    
    end

    covariancematrix = totalsum/n
    
    PDOP = sqrt(covariancematrix[1,1] + covariancematrix[2,2] + covariancematrix[3,3])
    PDOP_array[i,1] = PDOP
    off_actual = norm(mean_rescaled - r0)
    
    println("cov position erors: ", PDOP)
    println("Mean (m): ", mean_rescaled)
    println("Difference from Actual: ", off_actual)
    println("Iterations: ", iters)

end    

cov position erors: 133.55557812813768
Mean (m): [-6.173779936950038e6, -1.6019053855725604e6, -1.1075804907357538, 1.0390105815350062e-5]
Difference from Actual: 84.3150814412028
Iterations: 5
cov position erors: 132.2572692780192
Mean (m): [-6.173779007812833e6, -1.601904912643802e6, -1.1711491659418276, 1.038722331856539e-5]
Difference from Actual: 83.27695532574697
Iterations: 5
cov position erors: 135.7867665708171
Mean (m): [-6.17378236728235e6, -1.6019088537845432e6, -2.1363948916911237, 1.039861594223806e-5]
Difference from Actual: 88.23789847556776
Iterations: 5
cov position erors: 134.68585509251187
Mean (m): [-6.1737786239483e6, -1.6018987838936516e6, 0.5870237241625269, 1.038139282051737e-5]
Difference from Actual: 79.88197274010108
Iterations: 5
cov position erors: 132.06823702099024
Mean (m): [-6.173778835593852e6, -1.6019049186555059e6, -0.6824376809694032, 1.0384480397082548e-5]
Difference from Actual: 83.1281956738372
Iterations: 5
cov position erors: 130.9533997788424

cov position erors: 102.21179173036833
Mean (m): [-6.173777553832837e6, -1.6018817121565077e6, -0.11739001732880108, 1.0355700949497438e-5]
Difference from Actual: 72.37547210131939
Iterations: 4
cov position erors: 102.266720075767
Mean (m): [-6.173782149200982e6, -1.60187931223498e6, -0.025177779139523543, 1.0362152485093567e-5]
Difference from Actual: 76.18583205750606
Iterations: 4
cov position erors: 101.17662009670046
Mean (m): [-6.173779843896211e6, -1.6018805881854391e6, 0.13563074162769007, 1.0358912314497858e-5]
Difference from Actual: 74.27240420154273
Iterations: 4
cov position erors: 103.00089141763178
Mean (m): [-6.173783898618568e6, -1.6018826209808069e6, 0.16414103791530146, 1.0367894388696016e-5]
Difference from Actual: 78.72280867089903
Iterations: 4
cov position erors: 101.96348010952296
Mean (m): [-6.173781483630209e6, -1.601873572182825e6, -0.4435605379069773, 1.0356304819000743e-5]
Difference from Actual: 74.37114253396574
Iterations: 4
cov position erors: 103.620

cov position erors: 145.2195894737254
Mean (m): [-6.173797946419815e6, -1.601864301734889e6, 2.1470330676465665, 1.0390910487821031e-5]
Difference from Actual: 89.8710884979428
Iterations: 5


In [29]:
final_PDOP = PDOP_array[PDOP_array .!=0]

PDOP_max = findmax(final_PDOP)[1]
PDOP_min = findmin(final_PDOP)[1]

println("Maximum PDOP: ", PDOP_max)
println("Minimum PDOP: ", PDOP_min)

Maximum PDOP: 145.2195894737254
Minimum PDOP: 99.597453312329


UndefVarError: UndefVarError: setexcludinghandlers! not defined

UndefVarError: UndefVarError: setexcludinghandlers! not defined

UndefVarError: UndefVarError: setexcludinghandlers! not defined

UndefVarError: UndefVarError: setexcludinghandlers! not defined

...

UndefVarError: UndefVarError: setexcludinghandlers! not defined

UndefVarError: UndefVarError: setexcludinghandlers! not defined

UndefVarError: UndefVarError: setexcludinghandlers! not defined

UndefVarError: UndefVarError: setexcludinghandlers! not defined

UndefVarError: UndefVarError: setexcludinghandlers! not defined

In [ ]:
#write to a file
open("sat_data.txt", "w") do file
        writedlm(file, PDOP_array)
    end

In [ ]:
PDOP_filtered = filter(!iszero, PDOP_array)
n = size(PDOP_filtered)[1]
timesteps = range(1, n; length = n)
#RAANsep = zeros(n)
#RAANsep .= 12

plot(

    heatmap(

        x=timesteps,

        y=[12],

        z=PDOP_filtered'

    ),

    Layout(xaxis_side="top")

)

In [ ]:
#Equitorial Position
#In m
# x0= 5100.14e3 
# y0 = -5e3
# z0 = -1000e3
# t0 = 1e-5

# r0 = [x0,y0,z0,t0]

# Max_position = plot3d(eci_1[1,:], eci_1[2,:], eci_1[3,:], xlabel = "x", ylabel = "y", zlabel="z", title = "ISS Orbits")
# plot3d!(eci_3[1,:], eci_3[2,:], eci_3[3,:])
# scatter!([eci_1[1,500]], [eci_1[2,500]], [eci_1[3,500]], label ="satellite 1 end")
# scatter!([eci_2[1,500]], [eci_2[2,500]], [eci_2[3,500]], label ="satellite 2 end")
# scatter!([eci_3[1,500]], [eci_3[2,500]], [eci_3[3,500]], label ="satellite 3 end")
# scatter!([eci_4[1,500]], [eci_4[2,500]], [eci_4[3,500]], label ="satellite 4 end")

# scatter!([r0[1]], [r0[2]], [r0[3]], label ="Receiver Pose")

In [ ]:
# using PlotlyJS
# r0 = [-6.128804e6, -1.590206e6, 706.1502e3]

# sat1 = scatter(x=eci_1[1,:], y=eci_1[2,:], z=eci_1[3,:], type="scatter3d", mode="lines", name="orbit 1&2")
# sat3 = scatter(x=eci_3[1,:], y=eci_3[2,:], z=eci_3[3,:], type="scatter3d", mode="lines", name="orbit 3&4")
# satellites = scatter(x=[eci_1[1,1],eci_2[1,1], eci_3[1,1], eci_4[1,1]], y=[eci_1[2,1],eci_2[2,1], eci_3[2,1], eci_4[2,1]],z=[eci_1[3,1],eci_2[3,1], eci_3[3,1], eci_4[3,1]], mode="markers", marker_size = 4, type="scatter3d", name="satellites")
# tag = scatter(x = [r0[1], r0[1]], y = [r0[2], r0[2]], z = [r0[3], r0[3]], type="scatter3d", name="tag", mode="markers", marker_size=5)
# guess = scatter(x = [xyz[1], xyz[1]], y = [xyz[2], xyz[2]], z = [xyz[3], xyz[3]], type="scatter3d", name="guess", mode="markers", marker_size=5)

# #Plotting the sphere (Earth)
# n = 100
# u = range(-π, π; length = n)
# v = range(0, π; length = n)
# x = cos.(u) * sin.(v)'
# y = sin.(u) * sin.(v)' 
# z = ones(n) * cos.(v)' 

# earth = surface(z=z*6371000, x=x*6371000, y=y*6371000)

# plot([sat1,sat3, satellites, tag, earth, guess])

In [ ]:
currentguess = [-6069.837128422696e3, -1872.3495983074513e3, -576.2081281171355e3]
currentgoodguess = [-6122.192731259023, -1602.2259670043013, -795.1475980881328]
solution = [-6.12e6, -1.59e6, 707e3]
norm(currentguess-solution)*1e-3

In [ ]:
norm(solution-currentgoodguess)*1e-3

In [ ]:
print(R_EARTH)

In [ ]:
print(C_LIGHT)

In [ ]:
print(C_LIGHT/R_EARTH)

In [ ]:
initial_guess_test = [-6008.070607599407, -2138.8401454269697, -95.29951300518883]

In [ ]:
final_tag_position = [-6.128804e3, -1.590206e3, 706.1502]

In [ ]:
goodxyzguess = [-6171.613952212497, -1607.061707221621, -95.73653851577275]

In [ ]:
outloop = [-5.88796087165688e6, -1.6631186857729724e6, -1.801828127663494e6]*1e-3

In [ ]:
norm(final_tag_position - outloop)

In [ ]:
centroid_guess = [(eci_1[1,1]+eci_2[1,1]+eci_3[1,1]+eci_4[1,1])/4, (eci_1[2,1]+eci_2[2,1]+eci_3[2,1]+eci_4[2,1])/4, (eci_1[3,1]+eci_2[3,1]+eci_3[3,1]+eci_4[3,1])/4] 
#centroid_guess = eci_2[1:3, i]
onearth = sECEFtoGEOC(centroid_guess, use_degrees = true)
geodetic = [onearth[1], onearth[2], 0]

xyz_guess = sGEOCtoECEF(geodetic, use_degrees = true)#change to km
        
println("this is the initial guess!: ", xyz_guess)

In [ ]:
[-5.916954261422363e6, -1.5603343331057036e6, -1.798788655717042e6]*1e-3

In [ ]:
satposes = [eci_1[1,1] eci_1[2,1] eci_1[3,1];eci_2[1,1] eci_2[2,1] eci_2[3,1];eci_3[1,1] eci_3[2,1] eci_3[3,1];eci_4[1,1] eci_4[2,1] eci_4[3,1]]

In [ ]:
log(9.586691825001129)